# Web Scraping: Selenium

A menudo, los datos están disponibles públicamente para nosotros, pero no en una forma que sea fácilmente utilizable. Ahí es donde entra en juego el web scraping, podemos usar web scraping para obtener nuestros datos deseados en un formato conveniente que luego se puede usar. a continuación, mostraré cómo se puede extraer información de interés de un sitio web usando el paquete Selenium en Python. Selenium nos permite manejar una ventana del navegador e interactuar con el sitio web mediante programación. 

Selenium también tiene varios métodos que facilitan la extracción de datos.
En este Jupyter Notebook vamos a usar Python 3 en Windows.

En primer lugar, tendremos que descargar un controlador.

Usaremos ChromeDriver para Google Chrome. Para obtener una lista completa de controladores y plataformas compatibles, consulte [Selenium](https://www.selenium.dev/downloads/). Si desea utilizar Google Chrome, diríjase a [chrome](https://chromedriver.chromium.org/) y descargue el controlador que corresponde a su versión actual de Google Chrome.

Como saber cual es la version de chrome que utilizo simple utilizamos pegamos el siguiente enlace en la barra de chrome chrome://settings/help

Antes de comenzar se preguntaran si ya se BeautifulSoup cual es la diferencia con Selenium.

A diferencia BeautifulSoup, Selenium no trabaja con el texto fuente en HTML de la web en cuestión, sino que carga la página en un navegador sin interfaz de usuario. El navegador interpreta entonces el código fuente de la página y crea, a partir de él, un Document Object Model (modelo de objetos de documento o DOM). Esta interfaz estandarizada permite poner a prueba las interacciones de los usuarios. De esta forma se consigue, por ejemplo, simular clics y rellenar formularios automáticamente. Los cambios en la web que resultan de dichas acciones se reflejan en el DOM. La estructura del proceso de web scraping con Selenium es la siguiente:

URL → Solicitud HTTP → HTML → Selenium → DOM



## Comencemos importando las bibliotecas que usaremos:

In [1]:
from selenium import webdriver
import urllib3 # urllib3 es un cliente HTTP potente y fácil de usar para Python.
import re # Expresiones regulares 
import time
import pandas as pd

El objeto driver es con el que trabajaremos a partir de ahora

In [2]:
# especificamos el path hasta nuestro driver recién descargado:
chrome_driver_path = 'chromedriver.exe'
options  = webdriver.ChromeOptions()

In [3]:
# Creamos el driver con el que nos vamos a manejar en la sesión de scrapeo:
driver = webdriver.Chrome(executable_path = chrome_driver_path, options = options)

C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [4]:
# indicamos la URL de la página web a la que queremos acceder:
url = 'https://insolvencyinsider.ca/filing/'
# el objeto driver nos va a permitir alterar el estado del la página
driver.get(url)

Ahora si queremos hacer click en el boton de "Load more"..

Selenium proporciona varios métodos para localizar elementos en la página web. Usaremos el método find_element_by_xpath() para crear un objeto de botón, con el que luego podremos interactuar:

/html/body/div[2]/div/main/div/div/div/button

In [5]:
loadMore = driver.find_element_by_xpath(xpath="/html/body/div[2]/div/main/div/div/div/button")

C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """Entry point for launching an IPython kernel.


Antes de continuar, necesitaremos saber cuántas páginas hay para saber cuántas veces debemos hacer clic en el botón. Necesitaremos una forma de extraer el código fuente del sitio web. Afortunadamente, este proceso es relativamente sencillo con las bibliotecas urllib3 y re.

In [6]:
url = "https://insolvencyinsider.ca/filing/"
http = urllib3.PoolManager()
r = http.request("GET", url)
text = str(r.data)


```text``` ahora es una cadena. Ahora, necesitamos una forma de extraer total_pages de nuestra cadena de texto. Imprima texto para ver cómo podemos extraerlo usando RegEx con el paquete re. Podemos totalizar_páginas así:

In [7]:
totalPagesObj = re.search(pattern='"total_pages":\d+', string=text)
totalPagesStr = totalPagesObj.group(0)
totalPages = int((re.search(pattern="\d+", string=totalPagesStr)).group(0))

In [8]:
totalPages

113

El método de búsqueda toma un patrón y una cadena. En este caso nuestro patrón es '"total_pages":\d+' . Si no está familiarizado con RegEx, todo esto significa que estamos buscando la cadena "total_pages": con dos o más dígitos después de los dos puntos. \d se refiere a un dígito entre 0 y 9, mientras que + indica que Python debe buscar una o más de las expresiones regulares anteriores. Puedes leer más sobre el paquete re aquí. El método search() devuelve un objeto Match. re proporciona el método group() que devuelve uno o más subgrupos de la coincidencia. Pasamos 0 como argumento para indicar que queremos el parche completo. La tercera línea simplemente extrae el entero correspondiente a total_pages de la cadena.

In [9]:
print(totalPagesObj)
print(totalPagesStr)
print(totalPages)

<re.Match object; span=(116977, 116994), match='"total_pages":113'>
"total_pages":113
113


Con eso completo, ahora podemos cargar todas las páginas de Insolvency Insider. Podemos hacer clic en el botón Cargar más accediendo al método click() del objeto. Esperamos tres segundos entre clics para no sobrecargar el sitio web.

Recuerde que el total de las páginas son 88 pero comenzamos en 0 asi que es 88-1

In [10]:
for i in range(5):
    loadMore.click()
    time.sleep(3)

Una vez que ejecute esto, debería ver que se hace clic en el botón Cargar más y que se cargan las páginas restantes.
Una vez que se carga cada página, podemos comenzar a raspar el contenido. Ahora, eliminar ciertos elementos como el nombre de presentación, la fecha y la hiperreferencia es bastante sencillo. Podemos usar los métodos find_elements_by_class_name() y find_elements_by_xpath() de Selenium (importante la ```s``` extra después de element):

filing-name
filing-date
//*[@id='content']/div[2]/div/div[1]/h3/a

In [11]:
filingNamesElements = driver.find_elements_by_class_name("filing-name")

filingDateElements = driver.find_elements_by_class_name("filing-date")

filingHrefElements = driver.find_elements_by_xpath("//*[@id='content']/div[2]/div/div[1]/h3/a")

C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  """Entry point for launching an IPython kernel.
C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  """


También nos gustaría conocer los metadatos de presentación, es decir, el tipo de archivo, el sector de la empresa y el lugar en la que operan. Extraer estos datos requiere un poco más de trabajo.

//*[@id='content']/div[2]/div[%d]/div[2]/div[1]

In [12]:
filingMetas = []
for i in range(len(filingNamesElements) + 1):
    filingMetai = driver.find_elements_by_xpath(("//*[@id='content']/div[2]/div[%d]/div[2]/div[1]" %(i)))
    for element in filingMetai:
        filingMetaTexti = element.text
        filingMetas.append(filingMetaTexti)

C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  This is separate from the ipykernel package so we can avoid doing imports until


De cada elemento de la presentación de Metas podemos extraer el tipo de presentación, la industria y la provincia, así:

********

In [15]:
metaDict = {"Filing Type": [], "Industry": [], "Province": []}
for filing in filingMetas:
    filingSplit = filing.split("\n")
  
    for item in filingSplit:
        itemSplit = item.split(":")

        
        if itemSplit[0] == "Filing Type":
            metaDict["Filing Type"].append(itemSplit[1])
        elif itemSplit[0] == "Industry":
            metaDict["Industry"].append(itemSplit[1])
        elif itemSplit[0] == "Province":
            metaDict["Province"].append(itemSplit[1])
            
    if "Filing Type" not in filing:
        metaDict["Filing Type"].append("NA")
    elif "Industry" not in filing:
        metaDict["Industry"].append("NA")
    elif "Province" not in filing:
        metaDict["Province"].append("NA")

In [16]:
for key in metaDict:
    print(len(metaDict[key]))

60
60
60


*********

Ahora, todavía tenemos que poner nuestros nombres y fechas de presentación en las listas. Hacemos esto agregando el texto de cada elemento a una lista usando el método text() de antes:

In [17]:
filingName = []
filingDate = []
filingLink = []
# para cada elemento en la lista de elementos de nombre de archivo, agrega el
# texto del elemento a la lista de nombres de archivo.
for element in filingNamesElements:
    filingName.append(element.text)
# para cada elemento en la lista de elementos de la fecha de presentación, agrega el
# texto del elemento a la lista de fechas de presentación.
for element in filingDateElements:
    filingDate.append(element.text)
for link in filingHrefElements:
    if link.get_attribute("href"):
        filingLink.append(link.get_attribute("href"))

Una vez que tengamos eso, estamos listos para poner todo en un diccionario y luego crear un DataFrame de pandas:

In [18]:
# Crea un diccionario final con nombres y fechas de archivo.
fullDict = {
    "Filing Name": filingName,
    "Filing Date": filingDate, 
    "Filing Type": metaDict["Filing Type"],
    "Industry": metaDict["Industry"],
    "Province": metaDict["Province"],
    "Link": filingLink
}
# Crea un DataFrame.
df = pd.DataFrame(fullDict)
df["Filing Date"] = pd.to_datetime(df["Filing Date"], infer_datetime_format=True)

In [19]:
df

,Filing Name,Filing Date,Filing Type,Industry,Province,Link
0,GreenSpace Brands Inc.,2023-04-06,CCAA,Food & Accommodation,Ontario,https://insolvencyinsider.ca/filing/greenspace...
1,Phoena Holdings Inc. et al.,2023-04-04,CCAA,Cannabis,Ontario,https://insolvencyinsider.ca/filing/phoena-hol...
2,JMD-M Canada Inc.,2023-03-31,Receivership,Oil and Gas,Ontario,https://insolvencyinsider.ca/filing/jmd-m-cana...
3,RenoRun Inc.,2023-03-27,NOI,Transportation,Quebec,https://insolvencyinsider.ca/filing/renorun-inc/
4,FlexITy Solutions Inc. and FlexITy Holdings Inc.,2023-03-27,NOI,Technology,Ontario,https://insolvencyinsider.ca/filing/flexity-so...
5,Magna Gold Corp,2023-03-27,CCAA,Mining,Ontario,https://insolvencyinsider.ca/filing/magna-gold...
6,Field Trip Health & Wellness Ltd. et al.,2023-03-23,CCAA,Healthcare,Ontario,https://insolvencyinsider.ca/filing/field-trip...
7,"Radient Technologies Inc., Radient Technologie...",2023-03-21,Receivership,Cannabis,Alberta,https://insolvencyinsider.ca/filing/radient-te...
8,Canada Drives Ltd. et al.,2023-03-20,CCAA,Automotive,British Columbia,https://insolvencyinsider.ca/filing/canada-dri...
9,5684961 Manitoba Ltd.,2023-03-17,Receivership,Real Estate,Manitoba,https://insolvencyinsider.ca/filing/5684961-ma...


------------------------

# Ahora algo más visual

In [22]:
driver = webdriver.Chrome(executable_path = chrome_driver_path, options = options)

C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [23]:
# indicamos la URL de la página web a la que queremos acceder:
url = 'https://www.filmaffinity.com/es/main.html'
# el objeto driver nos va a permitir alterar el estado del la página
driver.get(url)

La página de Filmaffinity se ha abierto

Pero....

Nos hemos encontrado con un pop-up que nos pide aceptar cookies

1. Buscamos el botón
2. Hacemos click en el botón

Vamos a quitar el boton para seguir

In [24]:
elements_by_tag = driver.find_elements_by_tag_name('button')
elements_by_class_name = driver.find_elements_by_class_name('css-v43ltw')
element_by_xpath = driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[2]/div/button[2]')

C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  """Entry point for launching an IPython kernel.
C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  
C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  This is separate from the ipykernel package so we can avoid doing imports until


Una vez tenemos los elementos podemos hacer varias cosas con ellos

Podemos extraer todos los atributos que tenga

In [25]:
dir(element_by_xpath)
# obtenemos todos sus métodos y atributos:

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_execute',
 '_id',
 '_parent',
 '_upload',
 'accessible_name',
 'aria_role',
 'clear',
 'click',
 'find_element',
 'find_element_by_class_name',
 'find_element_by_css_selector',
 'find_element_by_id',
 'find_element_by_link_text',
 'find_element_by_name',
 'find_element_by_partial_link_text',
 'find_element_by_tag_name',
 'find_element_by_xpath',
 'find_elements',
 'find_elements_by_class_name',
 'find_elements_by_css_selector',
 'find_elements_by_id',
 'find_elements_by_link_text',
 'find_elements_by_name',
 'find_elements_by_partial_link_text',
 'find_elements_by_tag_name',
 'find_

Podemos evaluar que tipo de elemento es (tag)

In [26]:
element_by_xpath.tag_name

'button'

Podemos sacar el valor que tiene (el texto)

In [27]:
element_by_xpath.text

'ACEPTO'

In [28]:
for i in range(0,len(elements_by_tag)):
    print(elements_by_tag[i].text)

socios
MÁS OPCIONES
ACEPTO


Incluso podemos guardar una imagen del elemento

In [29]:
type(element_by_xpath)
# Vemos que es tipo 'WebElement' y en la documentación podremos encontrar sus métodos

selenium.webdriver.remote.webelement.WebElement

In [30]:
# guardamos como 'mi_imagen.png' la imagen asociada al xpath
element_by_xpath.screenshot('mi_imagen.png')

True

Evaluamos que elementos hemos encontrado por el tag:

In [31]:
for index, element in enumerate(elements_by_tag):
    print('Elemento:', index)
    print('Texto del elemento',index, 'es', element.text)
    print('El tag del elemento',index, 'es', element.tag_name)
    element.screenshot('mi_imagen'+str(index)+'.png')

Elemento: 0
Texto del elemento 0 es socios
El tag del elemento 0 es button
Elemento: 1
Texto del elemento 1 es MÁS OPCIONES
El tag del elemento 1 es button
Elemento: 2
Texto del elemento 2 es ACEPTO
El tag del elemento 2 es button


Basta de tonterias seguimos

Instanciamos el elemento del tag [2] en la variable boton aceptar

In [32]:
boton_aceptar = elements_by_tag[2]

Si el elemento es interactivo podremos hacer más cosas además de las anteriores. Por ejemplo: hacer click

In [33]:
boton_aceptar.click()

Buscamos una película por título

In [34]:
from selenium.webdriver.common.keys import Keys

/html/body/div[2]/div[1]/div/div[2]/form/div/input

In [ ]:
buscador = driver.find_element_by_xpath('')

In [ ]:
buscador.send_keys('')

In [ ]:
buscador.clear()

In [ ]:
# una vez escrita la búsqueda deberíamos poder activarla:
buscador.send_keys(Keys.ENTER)

In [ ]:
# volvemos a la página anterior
driver.back()

### Vamos a buscar todas las películas que se estrenan el próximo viernes

1. Cogemos los containers que hay en la zona lateral

In [35]:
menu_lateral = driver.find_element_by_id('lsmenu')
menu_lateral

C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """Entry point for launching an IPython kernel.


<selenium.webdriver.remote.webelement.WebElement (session="ede12887d1590d9b518cfdd2c7a949bb", element="81615749-eaad-41ff-8f57-81d43ae537d9")>

In [36]:
mis_secciones = menu_lateral.find_elements_by_tag_name('a')

c:\Users\lydia\AppData\Local\Programs\Python\Python37\lib\site-packages\selenium\webdriver\remote\webelement.py:359: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


2. Vemos con cuál nos tenemos que quedar

In [37]:
for a in mis_secciones:
    if a.text == 'Próximos estrenos':
        a.click()
        break

Accedemos al container central, en el que aparecen los estrenos por semana que queremos ver, exactamente igual que hemos hecho antes

In [38]:
cajon_central = driver.find_elements_by_id('main-wrapper-rdcat')

C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  """Entry point for launching an IPython kernel.


In [39]:
type(cajon_central)

list

In [46]:
driver = webdriver.Chrome(executable_path = chrome_driver_path, options = options)
# indicamos la URL de la página web a la que queremos acceder:
url = 'https://www.filmaffinity.com/es/rdcat.php?id=new_th_es#2023-04-05'
# el objeto driver nos va a permitir alterar el estado del la página
driver.get(url)
elements_by_tag = driver.find_elements_by_tag_name('button')
boton_aceptar = elements_by_tag[2]
boton_aceptar.click()
cajon_central = driver.find_elements_by_id('main-wrapper-rdcat')

C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.
C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  
C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  if __name__ == "__main__":


In [47]:
for semana in cajon_central:
    print(semana.find_element_by_tag_name('div').text)
    print(semana.find_element_by_tag_name('div').get_attribute('id'))

c:\Users\lydia\AppData\Local\Programs\Python\Python37\lib\site-packages\selenium\webdriver\remote\webelement.py:341: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


14 de abril de 2023
2023-04-14
5 de abril de 2023
2023-04-05
31 de marzo de 2023
2023-03-31
24 de marzo de 2023
2023-03-24
17 de marzo de 2023
2023-03-17
10 de marzo de 2023
2023-03-10
3 de marzo de 2023
2023-03-03


In [52]:
for semana in cajon_central:
    fecha = semana.find_element_by_tag_name('div').get_attribute('id')
    if fecha == '2023-04-05':
        break

Buscamos cómo acceder a las películas

In [53]:
caratulas = semana.find_elements_by_class_name('mc-poster')
lista_pelis = []
for peli in caratulas:
    lista_pelis.append(peli.find_element_by_tag_name('a').get_attribute('href'))

In [54]:
lista_pelis

['https://www.filmaffinity.com/es/film520586.html',
 'https://www.filmaffinity.com/es/film966677.html',
 'https://www.filmaffinity.com/es/film204461.html',
 'https://www.filmaffinity.com/es/film259000.html',
 'https://www.filmaffinity.com/es/film380291.html',
 'https://www.filmaffinity.com/es/film185176.html',
 'https://www.filmaffinity.com/es/film863159.html']

Una vez tenemos todas las urls vamos a ver qué hacemos con cada una de ellas

In [55]:
# Accedemos a la página de la primera pelicula
driver.get(lista_pelis[0])

Vamos a ver el proceso que deberíamos hacer con cada una de las películas:

1. Sacamos toda la información que nos interesa

In [56]:
# titulo, nota, numero de votos y ficha técnica
titulo = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/h1/span').text
nota = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[1]').text
votos = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[2]/span').text
ficha = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]')

C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  
C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  after removing the cwd from sys.path.
C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """


In [57]:
titulo

'Super Mario Bros: La película'

2. Creamos una lista a partir de la ficha técnica

In [58]:
# Los nombres estan con tag  = 'dt' y los valores con 'dd'
ficha_names = []
ficha_values = []

for name in ficha.find_elements_by_tag_name('dt'):
    ficha_names.append(name.text)
for value in ficha.find_elements_by_tag_name('dd'):
    ficha_values.append(value.text)

c:\Users\lydia\AppData\Local\Programs\Python\Python37\lib\site-packages\selenium\webdriver\remote\webelement.py:359: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


In [59]:
ficha_values

['The Super Mario Bros. Movie',
 '2023',
 '92 min.',
 ' Estados Unidos',
 'Aaron Horvath, Michael Jelenic',
 'Matthew Fogel. Personajes: Shigeru Miyamoto. Videojuego: Nintendo, Shigeru Miyamoto',
 'Kōji Kondō, Brian Tyler',
 'Animación',
 'Animación, Voz: Chris Pratt, Anya Taylor-Joy, Charlie Day, Jack Black, Keegan-Michael Key, Seth Rogen, Fred Armisen, Kevin Michael Richardson, Sebastian Maniscalco, Charles Martinet',
 'Coproducción Estados Unidos-Japón; Illumination Entertainment, Nintendo, Universal Pictures. Distribuidora: Universal Pictures',
 'Animación. Aventuras. Infantil. Fantástico. Comedia | Cine familiar. Videojuego',
 'Nintendo | Mario Bros. | Illumination Entertainment',
 'Mientras trabajan en una avería subterránea, los fontaneros de Brooklyn, Mario y su hermano Luigi, viajan por una misteriosa tubería hasta un nuevo mundo mágico. Pero, cuando los hermanos se separan, Mario deberá emprender una épica misión para encontrar a Luigi. Con la ayuda del champiñón local Toad y

3. Creamos un dataframe con la info

In [60]:
columns = ['Titulo', 'Nota', 'Votos']
columns.extend(ficha_names)
len(columns)

16

In [61]:
values = [titulo, nota, votos]
values.extend(ficha_values)
len(values)

16

In [62]:
pd.DataFrame([values],columns=columns)

,Titulo,Nota,Votos,Título original,Año,Duración,País,Dirección,Guion,Música,Fotografía,Reparto,Compañías,Género,Grupos,Sinopsis
0,Super Mario Bros: La película,"6,9",3.511,The Super Mario Bros. Movie,2023,92 min.,Estados Unidos,"Aaron Horvath, Michael Jelenic",Matthew Fogel. Personajes: Shigeru Miyamoto. V...,"Kōji Kondō, Brian Tyler",Animación,"Animación, Voz: Chris Pratt, Anya Taylor-Joy, ...",Coproducción Estados Unidos-Japón; Illuminatio...,Animación. Aventuras. Infantil. Fantástico. Co...,Nintendo | Mario Bros. | Illumination Entertai...,"Mientras trabajan en una avería subterránea, l..."


Ahora vamos a crear una función que nos haga todo esto para cada una de las películas:

In [63]:
def sacar_info(driver):
    
    titulo = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/h1/span').text
    try:
        nota = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]').text
        votos = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[2]').text
    except:
        nota = None
        votos = None
    ficha = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]')
    
    return titulo, nota, votos, ficha

def sacar_ficha(ficha):
    
    ficha_names = []
    ficha_values = []

    for name in ficha.find_elements_by_tag_name('dt'):
        ficha_names.append(name.text)
    for value in ficha.find_elements_by_tag_name('dd'):
        ficha_values.append(value.text)
        
    return ficha_names, ficha_values

def montar_df(ficha_names, ficha_values, titulo, nota, votos):
    
    columns = ['Titulo', 'Nota', 'Votos']
    columns.extend(ficha_names)
    values = [titulo, nota, votos]
    values.extend(ficha_values)
    
    return pd.DataFrame([values], columns = columns)
    
def nueva_pelicula(driver):
    
    titulo, nota, votos, ficha = sacar_info(driver)
    ficha_names, ficha_values = sacar_ficha(ficha)
    df_peli = montar_df(ficha_names, ficha_values, titulo, nota, votos)
    
    return df_peli


Vamos a ver cómo nos podemos mover entre ventanas del navegador

Abrir nueva ventana:

In [64]:
driver.execute_script('window.open("");')

Movernos a otra ventana

In [65]:
driver.switch_to.window(driver.window_handles[0])

Cerrar ventana

In [66]:
driver.close()

Una vez cerramos la ventana tenemos que indicarle a qué ventana tiene que ir

In [67]:
driver.switch_to.window(driver.window_handles[-1])

Sabiendo cómo podemos movernos por entre las ventanas y sabiendo cómo extraer de cada página toda la información que necesitamos vamos a crear nuestro dataframe:

In [68]:
# para abrir todos los links en lista_pelis
for link in lista_pelis:
    driver.execute_script('window.open("'+link+'");')
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(link)

In [69]:
# Creamos un dataframe con todas las pelis que se estrenan la próxima semana:
df_peliculas = pd.DataFrame()

for link in lista_pelis:
    driver.execute_script('window.open("");')
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(link)
    nueva_peli = nueva_pelicula(driver)
    df_peliculas = df_peliculas.append(nueva_peli)

C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """
C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  
C:\Users\lydia\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # Remove the CWD from sys.path while we load stuff.
c:\Users\lydia\AppData\Local\Programs\Python\Python37\lib\site-packages\selenium\webdriver\remote\webelement.py:359: UserW

In [70]:
df_peliculas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 0 to 0
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Titulo           7 non-null      object
 1   Nota             7 non-null      object
 2   Votos            7 non-null      object
 3   Título original  7 non-null      object
 4   Año              7 non-null      object
 5   Duración         7 non-null      object
 6   País             7 non-null      object
 7   Dirección        7 non-null      object
 8   Guion            7 non-null      object
 9   Música           6 non-null      object
 10  Fotografía       7 non-null      object
 11  Reparto          7 non-null      object
 12  Compañías        7 non-null      object
 13  Género           7 non-null      object
 14  Grupos           1 non-null      object
 15  Sinopsis         7 non-null      object
 16                   1 non-null      object
dtypes: object(17)
memory usage: 1008.0+ byt

In [71]:
df_peliculas

,Titulo,Nota,Votos,Título original,Año,Duración,País,Dirección,Guion,Música,Fotografía,Reparto,Compañías,Género,Grupos,Sinopsis,
0,Super Mario Bros: La película,"6,9\n3.512\nvotos",3.512\nvotos,The Super Mario Bros. Movie,2023,92 min.,Estados Unidos,"Aaron Horvath, Michael Jelenic",Matthew Fogel. Personajes: Shigeru Miyamoto. V...,"Kōji Kondō, Brian Tyler",Animación,"Animación, Voz: Chris Pratt, Anya Taylor-Joy, ...",Coproducción Estados Unidos-Japón; Illuminatio...,Animación. Aventuras. Infantil. Fantástico. Co...,Nintendo | Mario Bros. | Illumination Entertai...,"Mientras trabajan en una avería subterránea, l...",NaN
0,AIR,"7,1\n1.203\nvotos",1.203\nvotos,AIR,2023,112 min.,Estados Unidos,Ben Affleck,Alex Convery. Biografía sobre: Michael Jordan,NaN,Robert Richardson,"Matt Damon, Ben Affleck, Viola Davis, Jason Ba...","Amazon Studios, Skydance Productions, Mandalay...",Drama | Biográfico. Deporte. Años 80,NaN,Narra la increíble y revolucionaria asociación...,NaN
0,El exorcista del papa,"5,5\n631\nvotos",631\nvotos,The Pope's Exorcist,2023,103 min.,Estados Unidos,Julius Avery,"Evan Spiliotopoulos, Michael Petroni. Biografí...",Jed Kurzel,Khalid Mohtaseb,"Russell Crowe, Alex Essoe, Daniel Zovatto, Pet...","Screen Gems, 2.0 Entertainment, Jesus & Mary, ...",Terror | Biográfico. Religión. Posesiones / Ex...,NaN,"Película sobre Gabriele Amorth, un sacerdote q...",NaN
0,De Caperucita a loba,"4,4\n78\nvotos",78\nvotos,De Caperucita a loba,2023,92 min.,España,Chus Gutiérrez,Marta González de Vega,Karin Zielinski,Pilar Sánchez Díaz,"Marta González de Vega, José Mota, Berto Romer...","Bowfinger International Pictures, Esto También...",Comedia,NaN,Marta consigue pasar de Caperucita a loba cuan...,NaN
0,El inocente,"6,4\n316\nvotos",316\nvotos,L'innocentaka,2022,100 min.,Francia,Louis Garrel,"Louis Garrel, Tanguy Viel, Naïla Guiguet",Grégoire Hetzel,Julien Poupard,"Louis Garrel, Roschdy Zem, Anouk Grinberg, Noé...","Les Films des Tournelles, arte France Cinéma, ...",Comedia. Romance | Comedia romántica,NaN,"Cuando Abel descubre que su madre, Sylvie, que...",
0,Blue Jean,"7,0\n215\nvotos",215\nvotos,Blue Jean,2022,97 min.,Reino Unido,Georgia Oakley,Georgia Oakley,Chris Roe,Victor Seguin,"Rosy McEwen, Kerrie Hayes, Lydia Page, Lucy Ha...","BBC Film, BFI Film Fund, Kleio Films, Great Po...",Drama | Homosexualidad. Años 80,NaN,"En 1988, una profesora homosexual se ve enfren...",NaN
0,Empieza el baile,"7,0\n93\nvotos",93\nvotos,Empieza el baile,2023,98 min.,Argentina,Marina Seresesky,Marina Seresesky,Nicolás Guerschberg,Federico Rivares,"Darío Grandinetti, Mercedes Morán, Jorge Marra...",Coproducción Argentina-España; Meridional Prod...,Comedia. Drama | Comedia dramática. Vejez / Ma...,NaN,Carlos y Margarita fueron la pareja de tango m...,NaN


Ya tenemos un dataframe con todas las películas que se van a estrenar el próximo viernes